<H1>EJERCICIO #2 - EXAMEN FINAL</H1>
 <h3>Implementar una red neuronal, que permita realizar una regresión o clasificación de un problema que no podría ser resuelto por un simple perceptrón.<h3>
 <HR>
 <h3>
  NOMBRE: SAMUEL DAYLER AMONZABEL GONZALES <br>
  CARRERA: INGENIERIA DE SISTEMAS <BR>
  FECHA: 22/06/2023 <BR>
  <A>GITHUB: https://github.com/zohan22/SIS420_IA<A>
 <h3>

<p>En este ejemplo, vamos a utilizar un dataset de la pagina de kaggle el cual es el siguiente:<link><a> https://www.kaggle.com/datasets/kukuroo3/ body-performance-data</a></link> usaremos el framework de perceptron multicapao y el codigo establecido para redes neuronales para entrenarlo. La idea principal es ajustar los pesos de la red para minimizar.  Una vez entrenada la red, podremos utilizarla para hacer predicciones del rendimiento del cuerpo incorporando algunos datos.</p>

In [202]:
#Importamos librerias
#numeros aleatorios
import random
#utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# para el tratado de datos
import pandas as pd
# Modulo de optimizacion en scipy
from scipy import optimize
#Dar valores enteros a etiquetas
from sklearn.preprocessing import LabelEncoder
#para trabajar con fechas
from datetime import datetime

In [203]:
#Definimos la Clase MLP(perceptron multicapa -> secuencia de perceptrones) con 2 capas el inicializador y la llamada
class MLP:
    def __init__(self, layers):
        # el MLP es una lista de capas
        self.layers = layers

    def __call__(self, x):
        # calculamos la salida del modelo aplicando cada capa de manera secuencial que seran
        # los outputs de la capa siguiente
        for layer in self.layers:
            x = layer(x)
        return x

In [204]:
# Implementacion de las capas
class Layer():
  #lista de parametros y gradientes
    def __init__(self):
        self.params = []
        self.grads = []

    def __call__(self, x):
        # por defecto, devolvera los inputs
        # cada capa hará algo diferente aquí
        return x

    def backward(self, grad):
        # cada capa, calculará sus gradientes
        # y los devolverá para las capas siguientes
        return grad

    def update(self, params):
        # si hay parámetros, los actualizaremos
        # con lo que nos diga el optimizador
        return

In [205]:
#Este es el perceptron lineal con sus parametros
class Linear(Layer):
    def __init__(self, d_in, d_out):
        # pesos de la capa
        self.w = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(d_in+d_out)),
                                  size=(d_in, d_out))
        # bayas o unos
        self.b = np.zeros(d_out)

    def __call__(self, x):
        self.x = x
        self.params = [self.w, self.b]
        # salida del preceptrón que es el producto de la matriz mas el bayas
        return np.dot(x, self.w) + self.b

    def backward(self, grad_output):
        # gradientes para la capa siguiente (BACKPROP)
        grad = np.dot(grad_output, self.w.T) # multiplicar la matriz de pesos por el gradiente anterior
        self.grad_w = np.dot(self.x.T, grad_output)
        # gradientes para actualizar pesos
        self.grad_b = grad_output.mean(axis=0)*self.x.shape[0]
        self.grads = [self.grad_w, self.grad_b]
        return grad

    def update(self, params):
        self.w, self.b = params

<h3>Funciones de Activacion
</h3>
<h4>Relu</h4>

In [206]:
class ReLU(Layer):
    def __call__(self, x):
        self.x = x
        return np.maximum(0, x) #nos devuelve le valor maximo de las salidas

    def backward(self, grad_output):
        grad = self.x > 0
        return grad_output*grad
#expresion sigmoid (probabilidad entre 1 y 0)
def sigmoid(x):
  return 1 / (1 + np.exp(-x))
#expresion softmax un vector de sumatoria 1 que indica la probabilidad mas alta entre 1 y 0
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)
#implementacion
class Sigmoid(Layer):
    def __call__(self, x):
        self.x = x
        return sigmoid(x)
#derivada
    def backward(self, grad_output):
        grad = sigmoid(self.x)*(1 - sigmoid(self.x))
        return grad_output*grad

<h3>Optimizador</h3>
<h3>Descenso por el gradiente </h3>

In [207]:
#algoritmo para optimizar que solo es el descenso por el gradiente(otros Gradiente Conjugado,BFGS,L-BFGS )
class SGD():
    def __init__(self, net, lr):
        self.net = net
        self.lr = lr #LEARNING RATE->coeficiente de aprendizaje

    def update(self):
        for layer in self.net.layers:
            layer.update([
                params - self.lr*grads
                for params, grads in zip(layer.params, layer.grads)
            ])

<h3>Funciones de Perdida</h3>

In [208]:
class Loss():
    def __init__(self, net):
        self.net = net
#se encarga de desencadenar todo el calculo de las derivadas
    def backward(self):
        # derivada de la loss function con respecto
        # a la salida del MLP
        grad = self.grad_loss()
        # BACKPROPAGATION por todas las capas desde la ultima hasta la primera a lo contrario del FORWARD
        for layer in reversed(self.net.layers):
            grad = layer.backward(grad)
#Error Medio Cuadratico  para regresion
class MSE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = np.mean((self.output - self.target)**2)
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target
#Binary CrossEntropy para binario con sigmoid
class BCE(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target.reshape(output.shape)
        loss = - np.mean(self.target*np.log(self.output) - (1 - self.target)*np.log(1 - self.output))
        return loss.mean()

    def grad_loss(self):
        return self.output -  self.target
#multiclase con softmax para multiclase
class CrossEntropy(Loss):
    def __call__(self, output, target):
        self.output, self.target = output, target
        logits = output[np.arange(len(output)), target]
        loss = - logits + np.log(np.sum(np.exp(output), axis=-1))
        loss = loss.mean()
        return loss

    def grad_loss(self):
        answers = np.zeros_like(self.output)
        answers[np.arange(len(self.output)), self.target] = 1
        return (- answers + softmax(self.output)) / self.output.shape[0]

In [209]:
#Activamos el montaje de drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [210]:
#Cargamos nuestro dataset
data = pd.read_csv('/content/drive/MyDrive/DataSets/bodyPerformance.csv')
data

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


In [211]:
#mostramos la informacion de nuestro dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13393 entries, 0 to 13392
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      13393 non-null  float64
 1   gender                   13393 non-null  object 
 2   height_cm                13393 non-null  float64
 3   weight_kg                13393 non-null  float64
 4   body fat_%               13393 non-null  float64
 5   diastolic                13393 non-null  float64
 6   systolic                 13393 non-null  float64
 7   gripForce                13393 non-null  float64
 8   sit and bend forward_cm  13393 non-null  float64
 9   sit-ups counts           13393 non-null  float64
 10  broad jump_cm            13393 non-null  float64
 11  class                    13393 non-null  object 
dtypes: float64(10), object(2)
memory usage: 1.2+ MB


In [212]:
#verificamos que no haya duplicados
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
13388    False
13389    False
13390    False
13391    False
13392    False
Length: 13393, dtype: bool

In [213]:
#cambiamos los datos de tipo object a entero
columnas_categoricas = data.select_dtypes(include=['object']).columns

#Procesamos los caracteres o frases
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [214]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13393 entries, 0 to 13392
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      13393 non-null  float64
 1   gender                   13393 non-null  int64  
 2   height_cm                13393 non-null  float64
 3   weight_kg                13393 non-null  float64
 4   body fat_%               13393 non-null  float64
 5   diastolic                13393 non-null  float64
 6   systolic                 13393 non-null  float64
 7   gripForce                13393 non-null  float64
 8   sit and bend forward_cm  13393 non-null  float64
 9   sit-ups counts           13393 non-null  float64
 10  broad jump_cm            13393 non-null  float64
 11  class                    13393 non-null  int64  
dtypes: float64(10), int64(2)
memory usage: 1.2 MB


In [215]:
#mostramos los primeros 5 datos
print("Primeros 5 datos")
data.head()

Primeros 5 datos


,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,2
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,2
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1


In [216]:
print("Ultimos 5 datos")
data.tail()

Ultimos 5 datos


,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
13388,25.0,1,172.1,71.8,16.2,74.0,141.0,35.8,17.4,47.0,198.0,2
13389,21.0,1,179.7,63.9,12.1,74.0,128.0,33.0,1.1,48.0,167.0,3
13390,39.0,1,177.2,80.5,20.1,78.0,132.0,63.5,16.4,45.0,229.0,0
13391,64.0,0,146.1,57.7,40.4,68.0,121.0,19.3,9.2,0.0,75.0,3
13392,34.0,1,164.0,66.1,19.5,82.0,150.0,35.9,7.1,51.0,180.0,2


<p>Ya con los datos corregidos y convertidos y verificando que no haya duplicados procedemos a entrenar el modelo</p>

In [217]:
#cargamos los datos en las variables y los filtramos
x = data.iloc[:, :11]
y = data.iloc[:, 11]
m = y.size
print(x.shape)
print(y.shape)
print(x)
print("///"*30)
print(y)

(13393, 11)
(13393,)
        age  gender  height_cm  weight_kg  body fat_%  diastolic  systolic  \
0      27.0       1      172.3      75.24        21.3       80.0     130.0   
1      25.0       1      165.0      55.80        15.7       77.0     126.0   
2      31.0       1      179.6      78.00        20.1       92.0     152.0   
3      32.0       1      174.5      71.10        18.4       76.0     147.0   
4      28.0       1      173.8      67.70        17.1       70.0     127.0   
...     ...     ...        ...        ...         ...        ...       ...   
13388  25.0       1      172.1      71.80        16.2       74.0     141.0   
13389  21.0       1      179.7      63.90        12.1       74.0     128.0   
13390  39.0       1      177.2      80.50        20.1       78.0     132.0   
13391  64.0       0      146.1      57.70        40.4       68.0     121.0   
13392  34.0       1      164.0      66.10        19.5       82.0     150.0   

       gripForce  sit and bend forward_cm 

In [218]:
#Creamos un nuevo DataFrame con los datos modificados
nuevo_data = data.copy()

In [219]:
# Guardar el dataset actualizado en un nuevo archivo
nuevo_data.to_csv('bodyPerformance2.csv', index=False)

In [220]:
#cargamos con numpy
data = np.loadtxt("/content/bodyPerformance2.csv", delimiter=',',skiprows=1)
print(data)
x, y = data[:, :11].astype(int), data[:, 11].astype(int)
x = x.reshape(len(x),11)
print(x.shape)
print(y.shape)

[[ 27.    1.  172.3 ...  60.  217.    2. ]
 [ 25.    1.  165.  ...  53.  229.    0. ]
 [ 31.    1.  179.6 ...  49.  181.    2. ]
 ...
 [ 39.    1.  177.2 ...  45.  229.    0. ]
 [ 64.    0.  146.1 ...   0.   75.    3. ]
 [ 34.    1.  164.  ...  51.  180.    2. ]]
(13393, 11)
(13393,)


In [221]:
# #Normalización entre 0 y 1

X_mean, X_std = x.mean(axis=0), x.std(axis=0)
X_norm = (x - X_mean) / X_std
print(X_norm)

[[-0.71743212  0.76275036  0.45754957 ...  0.37999322  1.41695138
   0.67400898]
 [-0.8642197   0.76275036 -0.37247616 ...  0.1408761   0.92662893
   0.97501357]
 [-0.42385695  0.76275036  1.28757529 ... -0.33735814  0.64644468
  -0.22900479]
 ...
 [ 0.16329338  0.76275036  1.05042508 ...  0.1408761   0.36626042
   0.97501357]
 [ 1.99813815 -1.31104493 -2.62540312 ... -0.69603382 -2.78581245
  -2.88787869]
 [-0.20367558  0.76275036 -0.49105126 ... -0.93515094  0.78653681
  -0.25408851]]


In [222]:
#asignamos los datos de entrenamiento y de prueba
X_train, X_test, y_train, y_test = x[:800] , x[800:] , y[:800].astype(np.int64), y[800:].astype(np.int64)
print("DATOS PARA ENTRENAMIENTO")
X_train.shape , y_train.shape

DATOS PARA ENTRENAMIENTO


((800, 11), (800,))

In [223]:
print("DATOS PARA PRUEBA")
X_test.shape , y_test.shape

DATOS PARA PRUEBA


((12593, 11), (12593,))

In [224]:
#funciones de activacion
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [225]:
#Lineal: usada para regresión (junto a la función de pérdida MSE).
def linear(x):
    return x
#Sigmoid: usada para clasificación binaria (junto a la función de pérdida BCE).
def sigmoid(x):
  return 1 / (1 + np.exp(-x))
#Softmax: usada para clasificación multiclase (junto a la función de pérdida crossentropy, CE).
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

In [226]:
#Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

In [227]:
#derivadas
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [228]:
# clase base MLP

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2
    return self.activation(y_hat)

  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size]
            # salida del perceptrón
            y_pred = self(x)
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)
            # Backprop
            dldy = self.grad_loss(y, y_pred)
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [229]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

In [230]:
model = MLPClassification(D_in=11, H=100, D_out=4)
epochs, lr = 100, 0.2

# normalización datos
x_mean, x_std = X_train.mean(axis=0), X_train.std(axis=0)

# Calcula x_norm
x_norm = (X_train - x_mean) / x_std
model.fit(x_norm, y_train, epochs, lr, batch_size=10, log_each=10)

Epoch: 10/100, Loss: 0.88706
Epoch: 20/100, Loss: 0.82084
Epoch: 30/100, Loss: 0.78412
Epoch: 40/100, Loss: 0.75920
Epoch: 50/100, Loss: 0.73898
Epoch: 60/100, Loss: 0.72106
Epoch: 70/100, Loss: 0.70418
Epoch: 80/100, Loss: 0.68852
Epoch: 90/100, Loss: 0.67471
Epoch: 100/100, Loss: 0.66033


In [231]:
w = model.ws[-1]
w

(array([[-0.4825852 , -0.21893105,  0.07794902, ...,  0.52933576,
         -0.49391209, -0.49637815],
        [-0.08042926, -0.06098273, -0.21514689, ...,  0.52009941,
         -0.11583249,  0.29294213],
        [-0.0100136 , -0.37870433,  0.20837319, ..., -0.35134283,
         -0.29950203,  0.5510539 ],
        ...,
        [ 0.46257314, -0.30024413,  0.10692461, ...,  0.25052956,
         -0.174741  ,  0.70836709],
        [-0.8057032 , -0.54650145, -0.32168933, ..., -0.02325497,
         -1.10109223,  0.48986678],
        [-0.42110403,  0.14751589, -0.14967035, ..., -0.13895383,
         -1.08553038, -0.21250606]]),
 array([-0.03425377, -0.16309385, -0.34580921, -0.55983305, -0.66365815,
        -0.43855729, -0.31137926, -0.28608456, -0.33493995, -0.04086491,
        -0.32301734, -0.03464611, -0.25228887, -0.13466956, -0.62383511,
         0.00327897, -0.31802751, -0.1823967 , -0.12195716, -0.04837233,
        -0.1986991 , -0.91679223, -0.20184102, -0.87147367, -0.30020448,
        

In [232]:
print("Primeros 5 datos de entrada de prueba (X_test):")
print(X_test[:5])

print("Primeras 5 etiquetas de prueba (y_test):")
print(y_test[:5])

Primeros 5 datos de entrada de prueba (X_test):
[[ 45   0 149  45  30  88 121  20   6  22 138]
 [ 21   1 174  72  17  66 110  49  17  42 229]
 [ 30   0 161  67  16  81 127  44  25  39 222]
 [ 48   1 168  68  29  97 149  37   0  17 169]
 [ 29   0 163  58  28  82 115  26  23  44 165]]
Primeras 5 etiquetas de prueba (y_test):
[3 2 1 3 1]


In [233]:
x_new = X_test
X_new = [ 21, 0, 167.4, 72.18, 40, 82, 133, 18.7, -4, 21, 94]
y_pred = model.predict(w, X_new)
y_pred

array([-458.45973301,  -75.98615323,   -6.12418442,  546.52780552])